In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


Starting from here

In [ ]:
import pandas as pd
df_participants = pd.read_pickle('/content/drive/MyDrive/TD-BRAIN/TDBRAIN_participants_V2_data/df_participants.pkl')
print(f'all participants: {df_participants.shape}')

all participants: (370, 13)


In [ ]:
df_participants.sample(5)

,participants_ID,DISC/REP,indication,formal_status,Dataset,age,gender,sessID,nrSessions,EC,EO,diagnosis,diagnosis_group
1251,sub-88071949,DISCOVERY,MDD,UNKNOWN,NaN,60.26,1,1,1,True,True,MDD,MDD
323,sub-88005941,DISCOVERY,MDD,MDD,MDD-rTMS,21.02,0,1,1,True,True,MDD,MDD
1205,sub-88068481,DISCOVERY,MDD,UNKNOWN,NaN,72.60,1,1,1,True,True,MDD,MDD
769,sub-88032841,DISCOVERY,MDD,UNKNOWN,NaN,39.92,1,1,1,True,True,MDD,MDD
1020,sub-88052061,DISCOVERY,MDD,MDD,MDD-rTMS,19.34,1,1,1,True,True,MDD,MDD


In [ ]:
labels_df = df_participants[["participants_ID", "diagnosis"]]
labels_df

,participants_ID,diagnosis
245,sub-87974617,HEALTHY
246,sub-87974621,HEALTHY
247,sub-87974665,HEALTHY
248,sub-87974709,HEALTHY
249,sub-87974841,HEALTHY
...,...,...
1320,sub-88076941,MDD
1326,sub-88077345,MDD
1328,sub-88077525,MDD
1329,sub-88077569,MDD


In [ ]:
pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 68.9 MB/s eta 0:00:00


In [ ]:
import scipy
import os
import tensorflow as tf
import mne
import numpy as np

sample_ids = np.array(labels_df['participants_ID'])

ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg',\
           'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', \
           'eog', 'eog', 'eog', 'eog', 'ecg', 'eog', 'emg']

ch_names = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC3', 'FCz', 'FC4', 'T7', 'C3', 'Cz',
            'C4', 'T8', 'CP3', 'CPz', 'CP4', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'Oz', 'O2',
            'artifacts', 'VEOG', 'HEOG', 'Erbs', 'OrbOcc', 'Mass']

epoch_data_list = []
counter = 0
labels_samples_id = []
for sample_id in labels_df["participants_ID"]:
    # print(sample_id)
    npy_dir = f"/content/drive/MyDrive/TD-BRAIN/preprocessed/{sample_id}/ses-1/eeg/"
    if os.path.exists(npy_dir):
        for file in os.listdir(npy_dir):
            file_path = os.path.join(npy_dir, file)
            if os.path.isfile(file_path):
                if "EC" in file and '.npy' in file and 'BAD' not in file:
                  # eeg data
                  preprocessed_eeg = np.load(file_path, allow_pickle = True)
                  # preprocessed_eeg_data.append(preprocessed_eeg['data'])
                  info = mne.create_info(ch_names=ch_names[:32], ch_types=ch_types[:32], sfreq=500)
                  raw = mne.io.RawArray(np.squeeze(preprocessed_eeg['data']), info)
                  epoch_data = mne.make_fixed_length_epochs(raw, duration = 9.95, overlap = 0)
                  # labels
                  index = df_participants.loc[df_participants['participants_ID'] == sample_id].index
                  diagnosis = df_participants.loc[index, "diagnosis"]
                  diagnosis_value = [value for value in diagnosis][0]

                  for i in range(12):
                    epoch_data_list.append(epoch_data[i].get_data())
                    labels_samples_id.append([sample_id, diagnosis_value])
                    counter += 1
                  break

                  # print(f"Removing sample-id: {sample_id}. The eeg data for this participant is not available.")
        # break
epoch_data_array = np.stack(epoch_data_list)


Streaming output truncated to the last 5000 lines.
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=32, n_times=60008
    Range : 0 ... 60007 =      0.000 ...   120.014 secs
Ready.
Not setting metadata
12 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Using d

In [ ]:
counter

4356

In [ ]:
print(len(labels_samples_id))
print(labels_samples_id)

In [ ]:
print(epoch_data_array.shape)
labels_samples_array = np.array(labels_samples_id)
labels_samples_array.shape

(4356, 2)

In [ ]:
# Initialize an empty list to store data
data = []

# Loop over the array
for i in range(0, 4356, 12):
    # Append a tuple with participant ID and diagnosis to the list
    data.append((labels_samples_array[i][0], labels_samples_array[i][1]))

# Create the DataFrame from the list of tuples
labels_df = pd.DataFrame(data, columns=['participants_ID', 'diagnosis'])

# Display the DataFrame
# print(labels_df)

    participants_ID diagnosis
0      sub-87974617   HEALTHY
1      sub-87974621   HEALTHY
2      sub-87974665   HEALTHY
3      sub-87974709   HEALTHY
4      sub-87974841   HEALTHY
..              ...       ...
358    sub-88076941       MDD
359    sub-88077345       MDD
360    sub-88077525       MDD
361    sub-88077569       MDD
362    sub-88077613       MDD

[363 rows x 2 columns]


In [ ]:
file_path = "/content/drive/MyDrive/TD-BRAIN/training_data/data/unique_labels.csv"

# Save the DataFrame to CSV
labels_df.to_csv(file_path, index=False)

In [ ]:
save_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EC"
os.makedirs(save_dir, exist_ok=True)
np.save(os.path.join(save_dir, "labels_data.npy"), labels_samples_array)

In [ ]:
first_epoch_data = epoch_data[0].get_data()
print(epoch_data_array.shape)

Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
(4356, 1, 32, 4975)


In [ ]:
# Save EEG data and channels as NumPy arrays
save_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EC"
os.makedirs(save_dir, exist_ok=True)
np.save(os.path.join(save_dir, "raw_epoch_eeg_data.npy"), epoch_data_array)

Normalizing the Data

In [ ]:
from sklearn.preprocessing import StandardScaler

def scale_epoch_data(epoch_data_array):

    reshaped_data = epoch_data_array.reshape(-1, epoch_data_array.shape[-1])

    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(reshaped_data)


    scaled_epoch_data_array = scaled_data.reshape(epoch_data_array.shape)

    return scaled_epoch_data_array

In [ ]:
# Scale the epoch data array
scaled_epoch_data_array = scale_epoch_data(epoch_data_array)

In [ ]:
print(scaled_epoch_data_array[0])

[[[-0.06069194  0.08439861  0.01490577 ... -0.49181253 -0.3984212
   -0.26829051]
  [ 0.05329049  0.24674311  0.24831611 ... -0.16437871 -0.09824761
    0.00922452]
  [-0.13758785 -0.09469116 -0.24798399 ... -0.90770465 -0.70896085
   -0.42561074]
  ...
  [ 0.36793882  0.53954947  0.69323785 ... -1.75781728 -1.74378974
   -1.79445887]
  [-0.16168446 -0.62761097 -1.15416468 ... -0.22170755  0.05005584
    0.26467753]
  [ 0.34779039  0.37255668  0.26591169 ... -0.07446307  0.01592987
    0.06693167]]]


In [ ]:
# print(epoch_data_array[0])

In [ ]:
# Save EEG data and channels as NumPy arrays
save_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EC"
os.makedirs(save_dir, exist_ok=True)
np.save(os.path.join(save_dir, "normalized_epoch_eeg_data.npy"), epoch_data_array)